$\textbf{ROI slection}$<br>
$\text{In this workflow we crop the data temporaly and specialy. We also calculate the raw mc, pulse per ions and ion per pulse. }$

In [1]:
# Activate intractive functionality of matplotlib
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
from ipywidgets import fixed
from ipywidgets import interact_manual
from ipywidgets import widgets

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools, tof_tools

In [2]:
# Disable showing logging in Jupyter notebook
# logging.disable(sys.maxsize)

In [3]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [ ]:
# check pytables library and install it
# try:
#     import tables
# except:
#     !conda install --yes --prefix {sys.prefix} pytables

In [4]:
# create object for selection of instrument specifications of the dataset
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flightPathLength_d, t0_d, max_mc)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=54.0, description='t0:')

FloatText(value=400.0, description='Max mc:')

In [7]:
# Calculate the maximum possible time of flight (TOF)
max_tof = int(tof_tools.mc2tof(max_mc.value, 1000, 0, 0, flightPathLength_d.value))
print('The maximum possible TOF is:', max_tof, 'ns')
print('=============================')
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode.value
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/' + variables.dataset_name 
variables.result_data_name = 'cropped_' +  variables.dataset_name 
variables.result_path = dataset_main_path + '/' + variables.dataset_name + '/load_crop/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)

print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')

# Create data farame out of hdf5 file dataset
dld_group_storage = data_tools.load_data(dataset_path, tdc.value, mode='raw')

# Remove the data with tof greater thatn Max TOF or below 0 ns
data = data_tools.remove_invalid_data(dld_group_storage, max_tof)
print('Total number of Ions:', len(data))

The maximum possible TOF is: 11251 ns
The data will be saved on the path: D:/pyccapt/tests/data/data_TiO2
The dataset name after saving is: cropped_data_TiO2
The figures will be saved on the path: D:/pyccapt/tests/data/data_TiO2/load_crop/
{'dld': ['high_voltage', 'pulse', 'start_counter', 't', 'x', 'y']}
The number of data over max_tof: 2691
Total number of Ions: 160717


In [15]:
data

,high_voltage (V),pulse,start_counter,t (ns),x_det (cm),y_det (cm)
0,5300.0,0.0,0,0.000000,0.000000,0.000000
1,5300.0,0.0,0,6104.077148,-3.165880,-0.214491
2,5300.0,0.0,0,1758.948486,3.294220,0.566176
3,5300.0,0.0,0,1567.771362,-2.202440,-2.975180
4,5300.0,0.0,0,174.828705,-3.511540,0.861455
...,...,...,...,...,...,...
160712,5300.0,0.0,0,2536.992432,1.974480,3.047470
160713,5300.0,0.0,0,2904.909668,-3.127870,-1.704800
160714,5300.0,0.0,0,1256.020752,-3.252640,0.010828
160715,5300.0,0.0,0,4292.604492,-0.619795,3.199240


$\textbf {Temporal crop}$
<br>
<br>
$\text {Select the data by drawing a rectangel over the experiment history}$

In [16]:
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(False), data_crop=fixed(True), pulse=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='hist_ini'));


C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\797085356.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=11251.0, description='max_tof'), FloatText(value=1.0, description='frac'…

$\text{Boundaries of the selected(cropped) part of the graph is printed below}$

In [ ]:
# Plot and selected experiment history
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(True), data_crop=fixed(False), pulse=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='hist_rect'));


In [17]:
# Crop the dataset
print('Min Idx:', variables.selected_x1, 'Max Idx:', variables.selected_x2)
data_crop_t = data_loadcrop.crop_dataset(data, variables)

Min Idx: 609.0697823335213 Max Idx: 160342.94630825333


$\textbf {Spacial crop}$
<br>
<br>
$\text {Select the region of maximum concentration of Ions in the below plotted graph to utlize relevant data}$

In [24]:
# Plot and select the FDM
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
               draw_circle=fixed(False), data_crop=fixed(True), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm_ini'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\1567814332.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

$\text {The region selected in the previous step is displayed below.}$

In [25]:
# plot selected area in FDM
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
                draw_circle=fixed(True), data_crop=fixed(False), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm_circle'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\3314188428.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

In [28]:
# Crop the dataset
print('center x:', variables.selected_x_fdm, 'center y:', variables.selected_y_fdm)
print('Radios:', variables.roi_fdm)
if variables.roi_fdm > 0:
    data_crop_spatial = data_loadcrop.crop_data_after_selection(data_crop_t, variables)
else:
    print('select the data spacialy from cell below')

center x: -0.04289420003863542 center y: 0.04342140445797016
Radios: 3.761459173672527


d:\pyccapt\pyccapt\calibration\data_tools\data_loadcrop.py:306: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop.drop(np.where(mask_fdm)[0], inplace=True)


$\text {The final selected data after processing is shown below.}$

In [29]:
# Crop and plot the dataset
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_spatial), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
               draw_circle=fixed(False), data_crop=fixed(False), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\2530233233.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_spatial), variables=fixed(variables), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

$\text {Calculate pulses since the last event pulse and ions per pulse.}$

In [30]:
pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(data_crop_spatial)

# add two calculated array to the croped dataset
data_crop_spatial['pulse_pi'] = pulse_pi.astype(np.uintc)
data_crop_spatial['ion_pp'] = ion_pp.astype(np.uintc)


C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\949659944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop_spatial['pulse_pi'] = pulse_pi.astype(np.uintc)
C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\949659944.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop_spatial['ion_pp'] = ion_pp.astype(np.uintc)


$\text {The percentage of loss in ROI selection process.}$

In [31]:
# save the cropped data
print('tof Crop Loss {:.2f} %'.format((100 - (len(data_crop_spatial) / len(data)) * 100)))

tof Crop Loss 6.99 %


In [34]:
# exctract needed data from Pandas data frame as an numpy array
dld_highVoltage = data_crop_spatial['high_voltage (V)'].to_numpy()
dld_pulseVoltage = data_crop_spatial['pulse'].to_numpy()
dld_t = data_crop_spatial['t (ns)'].to_numpy()
dld_x = data_crop_spatial['x_det (cm)'].to_numpy()
dld_y = data_crop_spatial['y_det (cm)'].to_numpy()

In [35]:
def fine_tune_t_0(variables, t0_d, bin_size, mode, prominence, distance, percent, selector, plot, figname, lim, peaks_find_plot):
    variables.mc_calib = mc_tools.tof2mc(dld_t, t0_d, dld_highVoltage, dld_x, dld_y, flightPathLength_d.value, dld_pulseVoltage, mode='dc_voltage')
    tools.mc_hist_plot(variables, bin_size, mode, prominence, distance, percent, selector, plot, figname, lim, peaks_find_plot)
    
interact_manual(fine_tune_t_0, variables=fixed(variables), t0_d=widgets.FloatText(value=t0_d.value), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=fixed(True));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_35000\1170335243.py:5: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(fine_tune_t_0, variables=fixed(variables), t0_d=widgets.FloatText(value=t0_d.value), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=33.0, description='t0_d'), FloatText(value=0.1, description='bin_size'),…

In [36]:
data_crop_spatial_back = data_crop_spatial.copy()

In [37]:
data_crop_spatial_back.insert(0, 'x (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(1, 'y (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(2,'z (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(3,'mc_c (Da)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(4, 'mc (Da)', variables.mc_calib)
data_crop_spatial_back.insert(8,'t_c (ns)', np.zeros(len(dld_t)))

$\text {Remove the data with mc biger than max mc.}$

In [38]:
# Remove the data with mc biger than max mc
mask = (data_crop_spatial_back['mc (Da)'].to_numpy() > max_mc.value)
print('The number of data over max_mc:', len(mask[mask==True]))
data_crop_spatial_back.drop(np.where(mask)[0], inplace=True)
data_crop_spatial_back.reset_index(inplace=True, drop=True)

# Remove the data with x,y,t = 0
mask1 = (data_crop_spatial_back['x (nm)'].to_numpy() == 0)
mask2 = (data_crop_spatial_back['y (nm)'].to_numpy() == 0)
mask3 = (data_crop_spatial_back['t (ns)'].to_numpy() == 0)
mask = np.logical_and(mask1, mask2)
mask = np.logical_and(mask, mask3)
print('The number of data with having t, x, and y equal to zero is:', len(mask[mask==True]))
data_crop_spatial_back.drop(np.where(mask)[0], inplace=True)
data_crop_spatial_back.reset_index(inplace=True, drop=True)

The number of data over max_mc: 7486
The number of data with having t, x, and y equal to zero is: 6958


In [39]:
data_crop_spatial_back

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,0.0,6.680794,5300.0,0.0,0,0.0,665.327209,-0.621263,-3.556800,0,0
1,0.0,0.0,0.0,0.0,227.418041,5300.0,0.0,0,0.0,3746.363037,-3.116520,1.296670,0,0
2,0.0,0.0,0.0,0.0,335.491006,5300.0,0.0,0,0.0,4553.961426,-3.163760,-1.998820,0,0
3,0.0,0.0,0.0,0.0,217.798392,5300.0,0.0,0,0.0,3655.148193,-2.355530,-1.362360,0,0
4,0.0,0.0,0.0,0.0,50.423762,5300.0,0.0,0,0.0,1767.702148,0.808784,-1.789880,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135028,0.0,0.0,0.0,0.0,95.528361,5300.0,0.0,0,0.0,2435.999512,-1.069740,3.071310,0,0
135029,0.0,0.0,0.0,0.0,255.053233,5300.0,0.0,0,0.0,3931.918457,-0.522718,0.673361,0,0
135030,0.0,0.0,0.0,0.0,26.137510,5300.0,0.0,0,0.0,1285.479126,2.264250,-1.965060,0,0
135031,0.0,0.0,0.0,0.0,162.995464,5300.0,0.0,0,0.0,3165.060791,-0.215881,-2.668120,0,0


In [40]:
data_crop_spatial_back.dtypes

x (nm)              float64
y (nm)              float64
z (nm)              float64
mc_c (Da)           float64
mc (Da)             float64
high_voltage (V)    float64
pulse               float64
start_counter        uint32
t_c (ns)            float64
t (ns)              float64
x_det (cm)          float64
y_det (cm)          float64
pulse_pi             uint32
ion_pp               uint32
dtype: object

In [41]:
interact_manual(data_tools.save_data, data=fixed(data_crop_spatial_back), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

interactive(children=(Dropdown(description='hdf', options=(('True', True), ('False', False)), value=True), Dro…